# OpenPTV calibration using given markers and liboptv

In [2]:
# liboptv is different from openptv_python cause it's a C library with Cython bindings. requires different installation

In [3]:

import numpy as np
from scipy.optimize import minimize

import pathlib, os
working_path = pathlib.Path.cwd()

In [4]:
n_cams = 4

from optv.parameters import ControlParams, VolumeParams
cpar = ControlParams(n_cams)
cpar.read_control_par(b"parameters/ptv.par")


In [5]:
cpar.get_image_size()

(2560, 2048)

In [6]:

vpar = VolumeParams()
vpar.read_volume_par(b"parameters/criteria.par")
vpar.get_Zmin_lay(), vpar.get_Zmax_lay()

(array([-50., -50.]), array([1500., 1500.]))

In [ ]:
from optv.calibration import Calibration

b'calibration/cam4.tif'


In [8]:
cals = []
for i_cam in range(n_cams):
    cal = Calibration()
    tmp = cpar.get_cal_img_base_name(i_cam)
    print(tmp)
    cal.from_file(tmp + b".ori", tmp + b".addpar")
    print(cal.get_pos(), cal.get_angles())
    cals.append(cal)

b'calibration/cam1.tif'
[1908.5163676   159.83947734 3792.73650968] [ 0.0767755   0.43684216 -0.02824231]
b'calibration/cam2.tif'
[1918.50582672 2013.94616486 3790.84376551] [-0.54238883  0.38101067  0.22269959]
b'calibration/cam3.tif'
[-832.68057493  140.62429132 3808.29859692] [ 0.07130939 -0.39820728  0.02391263]
b'calibration/cam4.tif'
[-795.65582216 1982.45343091 3867.24986954] [-0.51564722 -0.33215712 -0.17554908]


In [15]:

from optv.transforms import convert_arr_metric_to_pixel
from optv.imgcoord import image_coordinates

for plane_z in [0, 350, 700, 1050, 1400]:

    # Save the modified data to a new file
    data = np.loadtxt(f'calibration/new_plane_{plane_z}.txt')
    targets = convert_arr_metric_to_pixel(
        image_coordinates(data[:,1:], cal, cpar.get_multimedia_params()),
    cpar,
    )

    # Combine targets and corresponding XYZ data
    combined_data = np.hstack((targets, data[:, 1:]))

    # Save to CSV file
    np.savetxt(f'calibration/targets_plane_{plane_z}.csv', combined_data, delimiter=',', header='x,y,X,Y,Z', comments='')


In [ ]:
def array_to_calibration(x:np.ndarray, cal:Calibration) -> None:
    cal.set_pos(x[:3])
    cal.set_angles(x[3:6])
    cal.set_primary_point(x[6:9])
    cal.set_radial_distortion(x[9:12])
    cal.set_decentering(x[12:14])
    cal.set_affine_trans(x[14:])
    return None

def calibration_to_array(cal:Calibration) -> np.ndarray:
    return np.concatenate([
        cal.get_pos(),
        cal.get_angles(),
        cal.get_primary_point(),
        cal.get_radial_distortion(),
        cal.get_decentering(),
        cal.get_affine(),
    ])

In [ ]:
def error_function(x, cal, XYZ, xy, cpar):
    
    array_to_calibration(x, cal)

    # print(np.concatenate([
    #     cal.get_pos(),
    #     cal.get_angles(),
    #     cal.get_primary_point(),
    #     cal.get_radial_distortion(),
    #     cal.get_decentering(),
    #     cal.get_affine(),
    # ]))
    
    targets = convert_arr_metric_to_pixel(
        image_coordinates(XYZ, cal, cpar.get_multimedia_params()),
    cpar,
    )
    # err = np.sum(np.abs(xy - targets))
    err = np.sum((xy - targets)**2)
    # print(err)
    return err

In [ ]:

x0 = calibration_to_array(cal)
print(x0)


In [ ]:
sol = minimize(error_function, x0, args=(cal, XYZ, xy, cpar), method='Nelder-Mead', tol=1e-11)

In [ ]:
sol.x



In [ ]:
array_to_calibration(sol.x, cal)

targets = convert_arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, cpar.get_multimedia_params()),
cpar,
)
four_points - targets


In [ ]:
targets = convert_arr_metric_to_pixel(
    image_coordinates(XYZ, cal, cpar.get_multimedia_params()),
cpar,
)


In [ ]:

import plotly.figure_factory as ff
# px.scatter(x=xy[:,0], y=xy[:,1], color=ID).show()
fig = ff.create_quiver(x=xy[:,0], y=xy[:,1], u=targets[:,0]-xy[:,0], v=targets[:,1]-xy[:,1], scale=5)
fig.show()


In [ ]:
# cal.write(working_path / "calibration" / "cam{cam_id}_scipy.ori", working_path / "calibration" / "cam{cam_id}_scipy.addpar")